In [1]:
#%%capture
%run ../dca/input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
root.gStyle.SetOptStat(0)

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


Error in <TUnixSystem::FindDynamicLibrary>: input/logo/PHENIXTools/lib/libLogoPainter.so does not exist in /home/yoren/bnl/ROOT/install/lib:.:/home/yoren/bnl/ROOT/install/lib:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/lib/x86_64-linux-gnu/tls/haswell/x86_64:/lib/x86_64-linux-gnu/tls/haswell:/lib/x86_64-linux-gnu/tls/x86_64:/lib/x86_64-linux-gnu/tls:/lib/x86_64-linux-gnu/haswell/x86_64:/lib/x86_64-linux-gnu/haswell:/lib/x86_64-linux-gnu/x86_64:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/usr/lib/x86_64-linux-gnu/tls/haswell/x86_64:/usr/lib/x86_64-linux-gnu/tls/haswell:/usr/lib/x86_64-linux-gnu/tls/x86_64:/usr/lib/x86_64-linux-gnu/tls:/usr/lib/x86_64-linux-gnu/haswell/x86_64:/usr/lib/x86_64-linux-gnu/haswell:/usr/lib/x86_64-linux-gnu/x86_64:/usr/lib/x86_64-linux-gnu:/lib/glibc-hwcaps/x86-64-v3:/lib/glibc-hwcaps/x86-64-v2:/lib/tls/haswell/x86_64:/lib/tls/haswell:/lib

In [2]:
iOptions = [2,2,5,9,10,16,15]
keffs = [1.0,1.0,1.0,0.99,0.98,0.995,0.953]
centralities = ["0%-93%"]
Opt_names = ["no ML eID","ML eID", "conv partner","tighter match","conv pattern", "ML conv rej", "ML hit, ddphi"]

In [3]:
N_centr = 6
N_options = 6
N_pt = 1
colors0=[1,2,root.kGreen+2,4,root.kCyan,root.kMagenta,root.kOrange+4,root.kGray,root.kGray,root.kBlack,root.kYellow,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
colors=[2,4,4,root.kGreen+2,root.kOrange+4,root.kMagenta,root.kGray,root.kGray,root.kBlack,root.kCyan,root.kOrange,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.4,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="../dca/input/data/"
file_names=["m_ee_DCA_531.root","m_ee_DCA_531.root","m_ee_DCA_532.root","m_ee_DCA_533.root","m_ee_DCA_59.root","m_ee_DCA_537.root","m_ee_DCA_65.root"]#,"m_ee_DCA_535.root"],"m_ee_DCA_536.root"]
hist_names = [["inv_mass_stat_00","inv_mass_syst_00",f"inv_mass_ee_DCA_V{iOption}_FG12",f"inv_mass_ee_DCA_V{iOption}_BG12"] for iOption in iOptions]
N_type = len(hist_names[0])

In [4]:
hists_read = []

for iFile in range(len(file_names)):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hists_emc_read0.append(infile.Get(hist_names[iFile][itype]))
        try: hists_emc_read0[-1].SetDirectory(root.nullptr); hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"_{iFile}")
        except: 
            print(f"Error setting directory for {hist_names[iFile][itype]}{0} in file {file_names[iFile]}")
            break
        hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"{iFile}")
    hists_read.append(hists_emc_read0)

    infile.Close()


Error setting directory for inv_mass_ee_DCA_V10_FG120 in file m_ee_DCA_59.root
Error setting directory for inv_mass_ee_DCA_V15_FG120 in file m_ee_DCA_65.root


In [5]:
infile = root.TFile.Open("../dca/input/Star.root", "READ")

d = infile.Get("Table 1")                  # TDirectoryFile
hists_star = d.Get("Graph1D_y1")           # TGraphAsymmErrors

# Optional safety check
if not hists_star:
    raise RuntimeError("Graph not found")

hists_star.SetName("STAR_AuAu")             # optional
hists_star.SetTitle("STAR Au+Au 0–80%")

infile.Close()

In [6]:

# ---- extract x values once ----
n = hists_star.GetN()
x = hists_star.GetX()
#  ---- set x errors from neighbor spacing ----
for i in range(n):
    if i == 0:
        dx = 0.5 * (x[1] - x[0])
    elif i == n - 1:
        dx = 0.5 * (x[n-1] - x[n-2])
    else:
        dx = 0.5 * (x[i+1] - x[i-1])

    hists_star.SetPointEXlow(i,  dx/2)
    hists_star.SetPointEXhigh(i, dx/2)


In [7]:
import ROOT as root

f = root.TFile.Open("../dca/input/Star200.root","READ")
d = f.Get("Table 13")

#d.ls()  # <-- shows the real object names in that directory

g = d.Get("Graph2D_y1;1")   # NOTE the ;1 (often required)
if not g:
    g = d.Get("Graph2D_y1") # fallback
if not g:
    raise RuntimeError("Can't find Graph2D_y1 in Table 13")

#c = root.TCanvas("c","c",900,650)
#g.Draw("P0")     # for TGraph2D-style scatter (what you see in your screenshot)
#c.Draw()         # important for notebook display


In [8]:
in_file = root.TFile.Open("output/sim_cocktail_massdep_corr.root","read")
in_file.ls()
sum_sim_proj_copycat = in_file.Get("sum_sim_proj_copycat;1")
sum_sim_proj_copycat.SetDirectory(root.nullptr)
in_file.Close()

TFile**		output/sim_cocktail_massdep_corr.root	
 TFile*		output/sim_cocktail_massdep_corr.root	
  KEY: TH1D	sum_sim_proj_copycat;1	


In [9]:
accepance_eff_low_mass = root.TF1("accepance_eff_low_mass","pol2",0,0.7)
accepance_eff_low_mass.SetParameters(2.7249856802644163e-05 , -7.603255066272503e-05 , 9.481844932778539e-05)
accepance_eff_high_mass = root.TF1("accepance_eff_high_mass","pol2",0.7,5.0 )
accepance_eff_high_mass.SetParameters(2.7249856802644163e-05 , -7.603255066272503e-05 , 9.481844932778539e-05)

for ibin in range(1,hists_read[len(file_names)-1][0].GetNbinsX()+1):#dividing hist1 adh hist2  by hist0
    mass = hists_read[len(file_names)-1][0].GetBinCenter(ibin)
    #if mass < 0.6:
    #    eff = accepance_eff_low_mass.Eval(0.6)
    #elif mass < 2.9:
    #    eff = accepance_eff_high_mass.Eval(mass)
    #else: eff = accepance_eff_high_mass.Eval(mass)
    eff = sum_sim_proj_copycat.GetBinContent(sum_sim_proj_copycat.FindBin(mass))
    if mass>2.3: eff = sum_sim_proj_copycat.GetBinContent(sum_sim_proj_copycat.FindBin(2.3))
    #print(f"mass: {mass}, eff: {eff}")
    if eff > 0:
        hists_read[len(file_names)-1][0].SetBinContent(ibin, hists_read[len(file_names)-1][0].GetBinContent(ibin)/eff)
        hists_read[len(file_names)-1][0].SetBinError(ibin, hists_read[len(file_names)-1][0].GetBinError(ibin)/eff)
        hists_read[len(file_names)-1][1].SetBinContent(ibin, hists_read[len(file_names)-1][1].GetBinContent(ibin)/eff)
        hists_read[len(file_names)-1][1].SetBinError(ibin, hists_read[len(file_names)-1][1].GetBinError(ibin)/eff)
    

In [10]:
dnch_phenix = 650
dnch_star  = 180*2
dnch_star200 = 750*2
ncoll_phenix = 260
ncoll_star200 = 350
iOpt = len(file_names)-1
hists_star.Scale(1./dnch_star)
hists_read[iOpt][0].Scale(1./dnch_phenix)
hists_read[iOpt][1].Scale(1./dnch_phenix)

In [11]:
# green markers: Centrality 20-40% (digitized)
eff_mass_raw = [
    0.0514, 0.1210, 0.1222, 0.1763, 0.2763,
    0.4019, 0.5120, 0.6615, 0.7715, 0.8911,
    1.0120, 1.0622, 1.1471, 1.3517, 1.6268,
    1.8768, 2.1017, 2.3505, 2.6519, 3.9007
]

eff_raw = [
    0.07919, 0.08142, 0.07363, 0.07728, 0.06358,
    0.06854, 0.06654, 0.07447, 0.07810, 0.07883,
    0.07477, 0.07875, 0.07919, 0.08348, 0.09110,
    0.08899, 0.09044, 0.09443, 0.09843, 0.09407
]
# bin edges from the table (variable width bins)
mass_edges = [
    0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10,
    0.15, 0.25, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05,
    1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 2.95, 3.00,
    3.05, 3.15, 3.25, 3.35, 3.75, 4.50, 5.00
]

# bin centers
mass_centers = [
    0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095,
    0.125, 0.200, 0.350, 0.500, 0.600, 0.700, 0.800, 0.900,
    1.000, 1.150, 1.375, 1.625, 1.875, 2.125, 2.375, 2.625,
    2.850, 2.975, 3.025, 3.100, 3.200, 3.300, 3.550, 4.125,
    4.750
]

# dN/dm_ee in PHENIX acceptance (GeV/c^2)^-1 from the table
yield_acc = [
    0.00478, 0.01769, 0.02320, 0.02107, 0.01643, 0.01288, 0.00949, 0.00653,
    0.00198, 0.00087, 0.000348, 0.00027, 0.00011, 0.000202, 0.000273, 0.000127,
    0.000124, 3.4e-05, 3.9e-05, 1.02e-05, 9.2e-06, 5.2e-06, 1.26e-06, 1.93e-06,
    4.0e-06, 6.8e-06, 9.0e-06, 1.38e-05, 5.8e-06, 1.38e-06, 3.60e-07, 9.3e-08,
    9.2e-08
]

# statistical uncertainties
stat_acc = [
    1.0e-04, 1.9e-04, 2.3e-04, 2.4e-04, 2.7e-04, 2.8e-04, 3.1e-04, 3.2e-04,
    1.3e-04, 1.0e-04, 8.7e-05, 1.0e-04, 7.0e-05, 7.0e-05, 1.3e-04, 3.3e-05,
    6.0e-05, 1.8e-05, 1.0e-05, 6.0e-06, 4.0e-06, 2.4e-06, 1.4e-06, 9.0e-07,
    8.0e-07, 1.4e-06, 1.3e-06, 9.0e-07, 7.0e-07, 6.0e-07, 1.9e-07, 7.0e-08,
    4.0e-08
]

# systematic uncertainties
# NOTE: a few bins in the screenshot are asymmetric; here I stored the *average* of up/down as a single sigma.
sys_acc = [
    5.0e-04, 1.7e-03, 2.3e-03, 2.05e-03, 1.65e-03, 1.35e-03, 9.0e-04, 6.0e-04,
    2.0e-04, 9.0e-05, 4.8e-05, 5.0e-05, 5.0e-05, 3.3e-05, 3.4e-05, 1.9e-05,
    2.25e-05, 7.5e-06, 4.5e-06, 1.4e-06, 9.5e-07, 5.0e-07, 1.35e-07, 1.9e-07,
    4.0e-07, 7.0e-07, 9.0e-07, 1.3e-06, 6.0e-07, 1.3e-07, 3.5e-08, 9.0e-09,
    9.0e-09
]
# efficiency evaluated at each mass_centers (linear interp; clamped outside [0.0514, 3.9007])
eff_at_centers = [
    0.07919, 0.07919, 0.07919, 0.079305345, 0.079625747, 0.079946149, 0.080266552, 0.080586954,
    0.073818909, 0.0740331, 0.066490446, 0.066757984, 0.071207826, 0.0757405, 0.078273955, 0.078531125,
    0.075172978, 0.079250806, 0.084125387, 0.091050142, 0.089005192, 0.090813662, 0.094755149, 0.098072999,
    0.097738363, 0.097301944, 0.097127377, 0.096865525, 0.09651639, 0.096167255, 0.095294417, 0.09407,
    0.09407
]

# corrected yield and uncertainties: (table)/eff_at_centers
yield_corr = [
    0.060361157, 0.22338679, 0.29296628, 0.26568197, 0.20634029, 0.16110845, 0.11823107, 0.081030485,
    0.026822396, 0.0117515, 0.0052338347, 0.0040444601, 0.001544774, 0.0026670011, 0.0034877502, 0.0016171932,
    0.0016495289, 0.00042901772, 0.00046359371, 0.00011202618, 0.00010336476, 5.7260107e-05, 1.329743e-05,
    1.9679219e-05, 4.0925588e-05, 6.9885551e-05, 9.2661825e-05, 0.00014246555, 6.009342e-05, 1.4349999e-05,
    3.7777659e-06, 9.8862549e-07, 9.7799511e-07
]

stat_corr = [
    0.0012627857, 0.0023992928, 0.0029044071, 0.0030262777, 0.003390863, 0.0035023575, 0.0038621318, 0.0039708661,
    0.0017610664, 0.0013507472, 0.0013084587, 0.0014979482, 0.00098303801, 0.00092420832, 0.0016608334, 0.00042021555,
    0.00079815915, 0.00022712703, 0.00011887018, 6.5897756e-05, 4.4941198e-05, 2.6427742e-05, 1.4774923e-05,
    9.1768377e-06, 8.1851176e-06, 1.4388202e-05, 1.3384486e-05, 9.2912313e-06, 7.2526542e-06, 6.2391299e-06,
    1.9938209e-06, 7.4412671e-07, 4.2521527e-07
]

sys_corr = [
    0.0063139285, 0.021467357, 0.029044071, 0.025849456, 0.020721941, 0.016886367, 0.011212641, 0.0074453738,
    0.0027093329, 0.0012156724, 0.00072190823, 0.00074897409, 0.00070217001, 0.00043569821, 0.00043437182,
    0.00024194229, 0.00029930968, 9.4636261e-05, 5.3491581e-05, 1.5376143e-05, 1.0673535e-05, 5.5057795e-06,
    1.4247247e-06, 1.9373324e-06, 4.0925588e-06, 7.1941009e-06, 9.2661825e-06, 1.3420667e-05, 6.2165607e-06,
    1.3518115e-06, 3.672828e-07, 9.5673435e-08, 9.5673435e-08
]


In [ ]:
sim_file_names = ['ccbar_soft_v22.root','bbbar_v03.root','thermal_300_v04.root','phi_25M_v04.root']
hists_sim = []
hist_select_3D_names = ["inv_mass_dca_gen_5","inv_mass_dca_gen_6","inv_mass_dca_gen_7","inv_mass_dca_gen_8","inv_mass_dca_gen_9"]
sim_file_path = "../sim/output/fall25/dca/"
for iFile in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(len(hist_select_3D_names)):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)
    infile.Close()

In [ ]:
for iFile in range(len(sim_file_names)):
    for ihist in range(1,len(hist_select_3D_names)): 
        hists_sim[iFile][0].Add(hists_sim[iFile][ihist])


hists_sim_porj_ccbar = hists_sim[0][0].ProjectionY(hists_sim[0][0].GetName()+f"_projYeadfsa",1,80,1,25)
hists_sim_porj_ccbar.SetDirectory(root.nullptr)
hists_sim_porj_bbbar = hists_sim[1][0].ProjectionY(hists_sim[1][0].GetName()+f"_projY",1,80,1,25)
hists_sim_porj_thermal = hists_sim[2][0].ProjectionY(hists_sim[2][0].GetName()+f"_projY",1,80,1,25)
hists_sim_porj_phi = hists_sim[3][0].ProjectionY(hists_sim[3][0].GetName()+f"_projY",1,80,1,25)
br =0.01
abs_norm_cc_new = 5000/3.5e7/10*br * 2
abs_norm_cc_new = 50.e6/324329759273*0.068**2/10 * 2. * 2.5
abs_norm_bb = 0.001102 * 10000 / 2104191 / 42 / 10 * 2
thermal_norm = 1.5e-7/0.156/3
hists_sim_porj_ccbar.Scale(1./hists_sim[0][0].GetEntries()*abs_norm_cc_new*10/hists_sim_porj_ccbar.GetBinWidth(1)/dnch_phenix)
hists_sim_porj_bbbar.Scale(1./hists_sim[1][0].GetEntries()*abs_norm_bb*10/hists_sim_porj_bbbar.GetBinWidth(1)/dnch_phenix)
hists_sim_porj_thermal.Scale(1./hists_sim[2][0].GetEntries()*thermal_norm*10/hists_sim_porj_thermal.GetBinWidth(1)/dnch_phenix)
hists_sim_porj_phi.Scale(1./hists_sim[3][0].GetEntries()*10/hists_sim_porj_phi.GetBinWidth(1)/dnch_phenix)

In [14]:
#removing nan bins IN hists_sim_porj_ccbar.GetBinContent(ibin)
for ibin in range(1,hists_sim_porj_ccbar.GetNbinsX()+1):
    if str(hists_sim_porj_ccbar.GetBinContent(ibin)) == 'nan':
        hists_sim_porj_ccbar.SetBinContent(ibin,0)
        hists_sim_porj_ccbar.SetBinError(ibin,0)

In [15]:
thermal_spectrum_change = root.TF1("thermal_spectrum_change","[0]*(x-[1])*(x-[1])+[2]",0,10)
thermal_spectrum_change.SetParameters(0.55,2.7,0.4)
for ibin in range(1,hists_sim_porj_thermal.GetNbinsX()+1):
    mass = hists_sim_porj_thermal.GetBinCenter(ibin)
    #mass = max(mass,0.7)
    hists_sim_porj_thermal.SetBinContent(ibin, hists_sim_porj_thermal.GetBinContent(ibin)*thermal_spectrum_change.Eval(mass))

In [16]:
infile = root.TFile.Open("../../pp_ee_analysis/sim/output/mass/prompt_v07.root", "read")
acceptance_hist = infile.Get("phenix_2arms_acceptance")
acceptance_proj = acceptance_hist.SetDirectory(root.nullptr)
infile.Close()

In [17]:
full_acceptance = acceptance_hist.ProjectionX(acceptance_hist.GetName()+f"_full",1,10,1,1)
phenix_acceptance = acceptance_hist.ProjectionX(acceptance_hist.GetName()+f"_phenix",1,10,2,2)
phenix_acceptance.Divide(full_acceptance)
c2 = root.TCanvas(f"c2",f"c2",400,400)
Format_Hist_total(phenix_acceptance,"m_{ee}, GeV","Acceptance",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
phenix_acceptance.Draw("")
phenix_acceptance.Rebin(1)
phenix_acceptance.Scale(1)
phenix_acceptance.Smooth(2)

star_acceptance = acceptance_hist.ProjectionX(acceptance_hist.GetName()+f"_star",1,10,4,4)
star_acceptance.Divide(full_acceptance)
star_acceptance.Rebin(1)
star_acceptance.Scale(1)
phenix_acceptance.SetMaximum(star_acceptance.GetMaximum()*1.2)
star_acceptance.Draw("same p")
c2.Draw()


ccbar_acceptance = acceptance_hist.ProjectionX(acceptance_hist.GetName()+f"_ccbar",1,10,3,3)
ccbar_acceptance.Divide(full_acceptance)

True

In [18]:
hbd_paper_edges = array('d', mass_edges)
hbd_paper = root.TH1D("hbd_paper", "Corrected; m_{ee} (GeV/c^{2}); (dN/dm) / #epsilon", len(mass_centers), hbd_paper_edges)

for i, (y, e, eff) in enumerate(zip(yield_acc, stat_acc,eff_at_centers), start=1):
    corr = phenix_acceptance.GetBinContent(phenix_acceptance.FindBin(hbd_paper.GetBinCenter(i)))+1e-14
    hbd_paper.SetBinContent(i, y/dnch_phenix/corr)
    hbd_paper.SetBinError(i, e/dnch_phenix/corr)   # stat error as TH1 error

In [19]:
f_200 = root.TFile.Open("../dca/input/Star200.root","READ")
g200 = f_200.Get("Table 13").Get("Graph2D_y1;1")
if not g200:
    g200 = f_200.Get("Table 13").Get("Graph2D_y1")
if not g200:
    raise RuntimeError("Graph2D_y1 not found")

n = g200.GetN()
f_200.Close

gr_200 = root.TGraphErrors(n)
for i in range(n):
    m  = float(g200.GetX()[i])  # mass
    acc = star_acceptance.GetBinContent(star_acceptance.FindBin(max(m,0.11)))+1e-14
    dm = float(g200.GetY()[i])  # dmass (usually bin width)
    val= float(g200.GetZ()[i]/700/2)  # dN/dmass
    val_err = float(g200.GetErrorZ(i)/700/2)  # error on dN/dmass
    gr_200.SetPoint(i, m, val/acc)
    gr_200.SetPointError(i, 0.5*dm, val_err/acc)  # x-error from bin width

c = root.TCanvas("c2","c2",900,650)
root.gPad.SetLogy()
gr_200.SetMarkerStyle(20)
gr_200.SetMarkerColor(root.kGreen+2)
gr_200.SetLineColor(root.kGreen+2)
gr_200.SetTitle("Table 13; mass; dN/dmass")
gr_200.Draw("AP")
c.Draw()


In [20]:
hbd_paper_ccbar_mass_points = array('d', [0.246,0.434,0.583,0.747,0.877,0.984,1.110,1.298,1.485,1.673,1.856,2.043,2.223,2.414,2.590,3.030,3.393])
hbd_paper_ccbar_yield_points = array('d', [0.00004544565369491261, 0.000038923695577546794, 0.00003837937157682314, 0.00004060342707443909, 0.000036277138871342626, 0.000029785583764372796, 0.00002247408179654268, 0.00001625583095235507, 0.000011114029869356516, 0.000007492344480411546, 0.0000050508435259769104, 0.0000032184388347708606, 0.0000022316385411283427, 0.000001483384793426641, 9.189730011909303e-7, 3.947573957486212e-7, 2.0653074397579281e-7])
hbd_paper_ccbar_mass_edges = array('d', [])
for i in range(len(hbd_paper_ccbar_mass_points)-1):
    hbd_paper_ccbar_mass_edges.append(0.5*(hbd_paper_ccbar_mass_points[i]+hbd_paper_ccbar_mass_points[i+1]))
hbd_paper_ccbar_mass_edges.insert(0, hbd_paper_ccbar_mass_points[0]-0.1*(hbd_paper_ccbar_mass_points[1]-hbd_paper_ccbar_mass_points[0]))
hbd_paper_ccbar_mass_edges.append(hbd_paper_ccbar_mass_points[-1]+0.1*(hbd_paper_ccbar_mass_points[-1]-hbd_paper_ccbar_mass_points[-2]))
hbd_paper_ccbar_hist = root.TH1D("hbd_paper_ccbar_hist","hbd_paper_ccbar_hist",len(hbd_paper_ccbar_mass_edges)-1,hbd_paper_ccbar_mass_edges)
hbd_paper_ccbar_hist_orig = root.TH1D("hbd_paper_ccbar_hist_orig","hbd_paper_ccbar_hist_orig",len(hbd_paper_ccbar_mass_edges)-1,hbd_paper_ccbar_mass_edges)
for i in range(len(hbd_paper_ccbar_mass_points)):
    corr = phenix_acceptance.GetBinContent(phenix_acceptance.FindBin(hbd_paper_ccbar_hist.GetBinCenter(i)))+1e-14
    hbd_paper_ccbar_hist.SetBinContent(i+1, hbd_paper_ccbar_yield_points[i]/dnch_phenix/corr)
    hbd_paper_ccbar_hist_orig.SetBinContent(i+1, hbd_paper_ccbar_yield_points[i]/ncoll_phenix)


In [21]:
tfile = root.TFile("input/newccbar6.root","READ")
my_new_ccbar_hist = tfile.Get("mee_phenix_phiacc_src")#mee_phenix_phiacc
my_new_ccbar_hist = my_new_ccbar_hist.ProjectionX(my_new_ccbar_hist.GetName()+f"_projX",3,3)
my_new_ccbar_hist.SetDirectory(root.nullptr)
my_new_ccbar_hist_eta = tfile.Get("mee_phenix_eta05_src")#mee_phenix_etaacc
my_new_ccbar_hist_eta = my_new_ccbar_hist_eta.ProjectionX(my_new_ccbar_hist.GetName()+f"_projX",3,3)
my_new_ccbar_hist_eta.SetDirectory(root.nullptr)
pt_phenix_eta_hist = tfile.Get("pt_phenix_eta_src")#pt_phenix_etaacc
pt_phenix_eta_hist = pt_phenix_eta_hist.ProjectionX(my_new_ccbar_hist.GetName()+f"_projX",3,3)
pt_phenix_eta_hist.SetDirectory(root.nullptr)
mee_star_hist = tfile.Get("mee_star_src")#mee_star
mee_star_hist = mee_star_hist.ProjectionX(my_new_ccbar_hist.GetName()+f"_projX",3,3)
mee_star_hist.SetDirectory(root.nullptr)
tfile.Close()

my_new_ccbar_hist.Scale(1./324329759273*0.068**2 * 2 * 1.5)
my_new_ccbar_hist.Scale(1./my_new_ccbar_hist.GetBinWidth(1))
my_new_ccbar_hist_eta.Scale(1./324329759273*0.068**2 * 2 * 1.5)
my_new_ccbar_hist_eta.Scale(1./my_new_ccbar_hist_eta.GetBinWidth(1))
pt_phenix_eta_hist.Scale(1./pt_phenix_eta_hist.GetBinWidth(1))
pt_phenix_eta_hist.Scale(1./324329759273*0.1/2)
mee_star_hist.Scale(1./mee_star_hist.GetBinWidth(1))    
mee_star_hist.Scale(1./324329759273*0.068**2 * 2 * 1.5 )

In [22]:
sim_pt_file_names = ['ccbar_soft_v17.root']
hists_pt_sim = []
hist_select_3D_names = ["hist_pt_orig","inv_mass_dca_gen_5"]
sim_file_path = "../sim/output/fall25/dca/"
for iFile in range(len(sim_pt_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_pt_file_names[iFile], "read")
    hists_pt_sim_file = []
    for ihist in range(len(hist_select_3D_names)):
        hists_pt_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_pt_sim_file[-1].SetDirectory(root.nullptr)
        hists_pt_sim_file[-1].SetName(hists_pt_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_pt_sim.append(hists_pt_sim_file)
    infile.Close()


In [23]:
mass_new_ccbar = hists_pt_sim[0][1].ProjectionY("mass_new_ccbar",1,80,1,25)
mass_new_ccbar_in_star = hists_pt_sim[0][1].ProjectionY("mass_new_ccbar_in_star",1,80,1,25)
mass_new_ccbar.Scale(1./hists_pt_sim[0][1].GetEntries()/771); mass_new_ccbar.Scale(1./mass_new_ccbar.GetBinWidth(1)); mass_new_ccbar.Scale(50.e6/324329759273*0.068**2 * 2 )
mass_new_ccbar_in_star.Scale(1./hists_pt_sim[0][1].GetEntries()/771); mass_new_ccbar_in_star.Scale(1./mass_new_ccbar_in_star.GetBinWidth(1)); mass_new_ccbar_in_star.Scale(50.e6/324329759273*0.068**2 * 2 )
for ibin in range(1,mass_new_ccbar.GetNbinsX()+1):
    mass = mass_new_ccbar.GetBinCenter(ibin)
    if my_new_ccbar_hist.GetBinContent(my_new_ccbar_hist.FindBin(mass)) > 0 and my_new_ccbar_hist_eta.GetBinContent(my_new_ccbar_hist_eta.FindBin(mass)) > 0:
        eff = my_new_ccbar_hist.GetBinContent(my_new_ccbar_hist.FindBin(mass)) / my_new_ccbar_hist_eta.GetBinContent(my_new_ccbar_hist_eta.FindBin(mass))
    mass_new_ccbar.SetBinContent(ibin, mass_new_ccbar.GetBinContent(ibin)*eff)
    if my_new_ccbar_hist_eta.GetBinContent(my_new_ccbar_hist_eta.FindBin(mass)) > 0 and mee_star_hist.GetBinContent(mee_star_hist.FindBin(mass)) > 0:
        eff_star = mee_star_hist.GetBinContent(mee_star_hist.FindBin(mass)) / my_new_ccbar_hist_eta.GetBinContent(my_new_ccbar_hist_eta.FindBin(mass))
    mass_new_ccbar_in_star.SetBinContent(ibin, mass_new_ccbar_in_star.GetBinContent(ibin)*eff_star)     

In [24]:
x=[0.055766793409378956,0.11660329531051963,0.20278833967046894,0.2788339670468948,0.3751584283903675,0.4562737642585551,0.5525982256020279,0.6489226869455006,0.7452471482889733,0.841571609632446,0.9277566539923954,1.0494296577946767,1.2015209125475284,1.3282636248415716,1.485424588086185,1.6273764258555132,1.7845373891001266,1.9366286438529783,2.0633713561470213,2.185044359949303,2.2965779467680605,2.4081115335868186,2.544993662864385,2.681875792141952,2.8136882129277563,2.9556400506970846,3.082382762991128,3.2344740177439797,3.3865652724968314,3.553865652724968,3.7414448669201517,3.9138149556400506,4.081115335868187]; y=[1.5243987765659025e-8,4.12119794103179e-8,8.595522875740646e-8,1.4133012458100373e-7,2.32379162999562e-7,2.947701945524981e-7,2.8846551130682425e-7,2.4795119013000296e-7,1.9974204114081995e-7,1.6090619681310152e-7,1.241357045886978e-7,9.786115307375474e-8,7.230276894396091e-8,5.458699684931828e-8,4.033051825545541e-8,2.8536391922827425e-8,2.1083546298861738e-8,1.4917942201895994e-8,1.0555387427926425e-8,8.321235121350755e-9,6.419655708082631e-9,5.171482072263617e-9,3.989688299910101e-9,2.9477019455249693e-9,2.4264789372131887e-9,1.7927557609349764e-9,1.4757543923599032e-9,1.1141620375362723e-9,8.055700036611764e-10,6.081875963262253e-10,4.493473239439193e-10,3.1794164568671033e-10,2.4003893439847446e-10]

# TH1D variable bins from midpoints between x's
edges_pp_2006=[x[0]-0.5*(x[1]-x[0])]+[0.5*(x[i]+x[i+1]) for i in range(len(x)-1)]+[x[-1]+0.5*(x[-1]-x[-2])]
ccbar_pp_2006=root.TH1D("ccbar_pp_2006",";x;y",len(x),array('d',edges_pp_2006))
for i,yi in enumerate(y,1): ccbar_pp_2006.SetBinContent(i,yi)

In [25]:
c5 = root.TCanvas("c5","c5",900,650)
root.gPad.SetLogy()
my_new_ccbar_hist.SetLineColor(root.kRed)
my_new_ccbar_hist.SetMaximum(max(my_new_ccbar_hist.GetMaximum(),ccbar_pp_2006.GetMaximum(),mass_new_ccbar.GetMaximum())*1.2)
my_new_ccbar_hist.Draw("hist")
#hbd_paper_ccbar_hist_orig.Draw("same hist")
ccbar_pp_2006.Draw("same hist")
mass_new_ccbar.SetLineColor(root.kMagenta+2)
mass_new_ccbar.Draw("same hist")
print(f"Integral of my_new_ccbar_hist: {my_new_ccbar_hist.Integral(my_new_ccbar_hist.FindBin(0.4),my_new_ccbar_hist.FindBin(4.4))}, Integral of ccbar_pp_2006: {ccbar_pp_2006.Integral(ccbar_pp_2006.FindBin(0.4),ccbar_pp_2006.FindBin(4.4))}")
c5.Draw()

Integral of my_new_ccbar_hist: 7.96600335589411e-06, Integral of ccbar_pp_2006: 1.9246047062846997e-06


In [26]:
import math
pp200_pt=[1.0860262008733625, 1.2877729257641923, 1.5078602620087338, 1.7096069868995634, 1.8930131004366815, 2.085589519650655, 2.3056768558951966, 2.5074235807860266, 2.7, 2.8925764192139742, 3.1034934497816598, 3.2960698689956334, 3.4978165938864634, 3.681222707423581, 3.882969432314411, 4.258951965065503, 4.744978165938865, 5.4969432314410485, 6.487336244541485, 7.5052401746724895, 8.495633187772926]; pp200_yiled=[0.00028109619171294, 0.00013154791513423709, 0.00006331793133551688, 0.00003134607772368954, 0.00001641599365505144, 0.000008358674329351349, 0.000004377451812785282, 0.000002425117717019789, 0.00000146179171526938, 8.566914399960481e-7, 5.311177791975002e-7, 3.292738577859326e-7, 1.9297283767113261e-7, 1.3388057140023308e-7, 9.030778509834389e-8, 4.34678279019821e-8, 1.7183613816327383e-8, 5.424392977339485e-9, 1.367344230883996e-9, 4.080252343563833e-10, 1.4413804456678368e-10]
pp200_pt=[1.1221922731356693,1.2587601078167117,1.4528301886792452,1.6612758310871518,1.8697214734950585,2.063791554357592,2.257861635220126,2.4231805929919137,2.566936208445642,2.7322551662174304,2.911949685534591,3.106019766397125,3.422282120395328,3.6091644204851754,3.903863432165319,4.162623539982031,4.435759209344115,4.665768194070081,4.996406109613657,5.327044025157233,5.8517520215633425,6.4914645103324355,7.044923629829291,7.849955076370171,8.31716082659479];  pp200_yiled=[0.00012138917811578232, 0.00007111469657803517, 0.000032971767983075646, 0.000015287100082566523, 0.000008101457069664842, 0.0000049074548595132095, 0.0000025152464894487903, 0.000001575387945920774, 0.0000011278455676914407, 7.304161915285097e-7, 4.891081896038604e-7, 3.167566121903624e-7, 1.678663763128751e-7, 1.0871374839297723e-7, 6.159565425630805e-8, 4.1246263829013564e-8, 2.416373174864189e-8, 1.4494972973237007e-8, 1.1203323576684991e-8, 4.563360665162244e-9, 2.253394438147941e-9, 1.008410662405532e-9, 5.731145230827419e-10, 2.8368096047762684e-10, 1.6072957302064741e-10]

# Convert from (1/(2π pT)) dσ/dpT  ->  dσ/dpT, then /42
pp200_true_yiled=[(2*math.pi*pp200_pt[i]*pp200_yiled[i])/42.0 for i in range(len(pp200_pt))]

# Variable-bin edges from midpoints
edges=[pp200_pt[0]-0.5*(pp200_pt[1]-pp200_pt[0])]+[0.5*(pp200_pt[i]+pp200_pt[i+1]) for i in range(len(pp200_pt)-1)]+[pp200_pt[-1]+0.5*(pp200_pt[-1]-pp200_pt[-2])]
pp200_charm=root.TH1D("pp200_opencahrmelectrons_pt_hist",";p_{T} (GeV/c);(d#sigma/dp_{T})/42",len(pp200_pt),array('d',edges))
for i,yi in enumerate(pp200_true_yiled,1): pp200_charm.SetBinContent(i,yi)

In [27]:
hists_pt_sim_proj = hists_pt_sim[0][0].ProjectionX(hists_pt_sim[0][0].GetName()+f"_projXeadfsa",1,5,1,2)
hists_pt_sim_proj.SetLineColor(root.kRed)
hists_pt_sim_proj.Scale(1./50e6*50e6*0.068/324329759273/hists_pt_sim_proj.GetBinWidth(1)*10)

In [28]:
c6 = root.TCanvas("c6","c6",900,650)
root.gPad.SetLogy()
#hists_pt_sim_proj.Scale(0.33)
hists_pt_sim_proj.Draw("hist")
pp200_charm.Draw("same hist")
pt_phenix_eta_hist.SetLineColor(root.kMagenta)
pt_phenix_eta_hist.Scale(1)             
pt_phenix_eta_hist.Draw("same hist")
c6.Draw()

In [29]:
for ibin in range(1,hists_pt_sim_proj.GetNbinsX()+1):#divide manually pp200_charm by hists_pt_sim_proj
    sim_val = hists_pt_sim_proj.GetBinContent(ibin)
    charm_val = pp200_charm.GetBinContent(pp200_charm.FindBin(hists_pt_sim_proj.GetBinCenter(ibin)))
    if sim_val > 0:
        hists_pt_sim_proj.SetBinContent(ibin, charm_val/sim_val)
    else:       hists_pt_sim_proj.SetBinContent(ibin, 0)
    hists_pt_sim_proj.SetBinError(ibin,0.2)
pt_phenix_eta_hist.Rebin(2)
pt_phenix_eta_hist.Scale(0.5)
for ibin in range(2,pt_phenix_eta_hist.GetNbinsX()+1):#divide manually pp200_charm by pt_phenix_eta_hist

    pt_phenix_eta_val = pt_phenix_eta_hist.GetBinContent(ibin)
    charm_val = pp200_charm.GetBinContent(pp200_charm.FindBin(pt_phenix_eta_hist.GetBinCenter(ibin)))

    if pt_phenix_eta_val > 0:
        pt_phenix_eta_hist.SetBinContent(ibin, (charm_val/pt_phenix_eta_val+1)/2)
    else:       pt_phenix_eta_hist.SetBinContent(ibin, 0)
    pt_phenix_eta_hist.SetBinError(ibin,0.2)

In [30]:
c6 = root.TCanvas("c6","c6",1000,500)
c6.Divide(2,1)
c6.cd(1)
hists_pt_sim_proj.Draw("hist")
hists_pt_sim_proj.GetYaxis().SetRangeUser(0.5,3)
fit_func_pt = root.TF1("fit_func_pt","[0]+[1]/x",0,10)
fit_func_pt.SetParameters(1,1,-0.1)
#fit_func_pt.FixParameter(0,1)
hists_pt_sim_proj.Fit(fit_func_pt,"QR","",1,4)
c6.cd(2)
pt_phenix_eta_hist.Draw("hist")
pt_phenix_eta_hist.GetYaxis().SetRangeUser(0.5,3)
fit_func_pt1 = root.TF1("fit_func_pt1","[0]+[1]/x",0,10)
fit_func_pt1.SetParameters(1,1,-0.1)
#fit_func_pt1.FixParameter(0,1)
pt_phenix_eta_hist.Fit(fit_func_pt1,"QR","",1,8)
c6.Draw()
print(f"const double new_ccbar_pt_params[3] =  {{ {fit_func_pt.GetParameter(0)}, {fit_func_pt.GetParameter(1)}, {fit_func_pt.GetParameter(2)} }}")
print(f"const double new_ccbar_pt_eta_params[3] =  {{ {fit_func_pt1.GetParameter(0)}, {fit_func_pt1.GetParameter(1)}, {fit_func_pt1.GetParameter(2)} }}")

const double new_ccbar_pt_params[3] =  { 0.8068162417599091, 1.1593055140420894, nan }
const double new_ccbar_pt_eta_params[3] =  { 0.7042617060886011, 0.2089226631664785, nan }


In [31]:
star200_paper_ccbar_mass_points = array('d', [0.197, 0.304, 0.422, 0.557, 0.679, 0.814, 0.960, 1.118, 1.296, 1.462, 1.644, 1.822, 1.996, 2.162, 2.387, 2.656, 2.857, 3.142])
star200_paper_ccbar_yield_points = array('d', [ 0.000011998139968649307, 0.00001629391012439856, 0.000022616759492228645, 0.000026938893095901753, 0.000026356399145461657, 0.000021649259091974006, 0.000015941590374559995, 0.000010523298721084975, 0.000006505656657695662, 0.000004021891772683256, 0.0000023285662984981964, 0.0000012905044120433, 8.154407395185154e-7, 4.619095719403757e-7, 2.3974349678010785e-7, 1.0446897695733761e-7, 5.789728889061239e-8, 2.5786500337705836e-8])
star200_paper_ccbar_mass_edges = array('d', [])
for i in range(len(star200_paper_ccbar_mass_points)-1):
    star200_paper_ccbar_mass_edges.append(0.5*(star200_paper_ccbar_mass_points[i]+star200_paper_ccbar_mass_points[i+1]))
star200_paper_ccbar_mass_edges.insert(0, star200_paper_ccbar_mass_points[0]-0.1*(star200_paper_ccbar_mass_points[1]-star200_paper_ccbar_mass_points[0]))
star200_paper_ccbar_mass_edges.append(star200_paper_ccbar_mass_points[-1]+0.1*(star200_paper_ccbar_mass_points[-1]-star200_paper_ccbar_mass_points[-2]))
star200_paper_ccbar_hist = root.TH1D("star200_paper_ccbar_hist","star200_paper_ccbar_hist",len(star200_paper_ccbar_mass_edges)-1,star200_paper_ccbar_mass_edges)
star200_paper_ccbar_hist_orig = root.TH1D("star200_paper_ccbar_hist_orig","star200_paper_ccbar_hist_orig",len(star200_paper_ccbar_mass_edges)-1,star200_paper_ccbar_mass_edges)
for i in range(len(star200_paper_ccbar_mass_points)):
    corr = star_acceptance.GetBinContent(star_acceptance.FindBin(star200_paper_ccbar_hist.GetBinCenter(i)))+1e-14
    star200_paper_ccbar_hist.SetBinContent(i+1, star200_paper_ccbar_yield_points[i]/dnch_star200/corr*127)
    star200_paper_ccbar_hist_orig.SetBinContent(i+1, star200_paper_ccbar_yield_points[i]/ncoll_star200*127)

In [32]:
c7 = root.TCanvas("c7","c7",900,650)
root.gPad.SetLogy()
mee_star_hist.SetLineColor(root.kRed)
mee_star_hist.SetMaximum(max(mee_star_hist.GetMaximum(),star200_paper_ccbar_hist_orig.GetMaximum())*1.2)
mee_star_hist.Draw("hist")
star200_paper_ccbar_hist_orig.Draw("same hist")
mass_new_ccbar_in_star.SetLineColor(root.kMagenta+2)
mass_new_ccbar_in_star.Draw("same hist")
c7.Draw()

In [33]:
ratio_ccbar_mine_hbd = []; mass_ratio_ccbar = []
for ibin in range(1,hbd_paper_ccbar_hist.GetNbinsX()+1):
    mass = hbd_paper_ccbar_hist.GetBinCenter(ibin)
    bin_hbd = hists_sim_porj_ccbar.FindBin(mass)
    if bin_hbd >0 and hists_sim_porj_ccbar.GetBinContent(bin_hbd) >0:
        ratio = hbd_paper_ccbar_hist.GetBinContent(ibin)/hists_sim_porj_ccbar.GetBinContent(bin_hbd)
    else:
        ratio = 0
    ratio_ccbar_mine_hbd.append(ratio)
    mass_ratio_ccbar.append(mass)

ratio_ccbar_mine_star200 = []; mass_ratio_star200 = []
for ibin in range(1,star200_paper_ccbar_hist.GetNbinsX()+1):
    mass = star200_paper_ccbar_hist.GetBinCenter(ibin)
    bin_hbd = hists_sim_porj_ccbar.FindBin(mass)
    if bin_hbd >0 and hists_sim_porj_ccbar.GetBinContent(bin_hbd) >0:
        ratio = star200_paper_ccbar_hist.GetBinContent(ibin)/hists_sim_porj_ccbar.GetBinContent(bin_hbd)
    else:
        ratio = 0
    ratio_ccbar_mine_star200.append(ratio)
    mass_ratio_star200.append(mass)
#print("Ratio of ccbar yield from my calcuation to HBD paper values:",ratio_ccbar_mine_hbd)

if True:
    for ibin in range(1,hists_sim_porj_ccbar.GetNbinsX()+1):
        hists_sim_porj_ccbar.SetBinContent(ibin, hists_sim_porj_ccbar.GetBinContent(ibin)/(ccbar_acceptance.GetBinContent(ccbar_acceptance.FindBin(hists_sim_porj_ccbar.GetBinCenter(ibin)))+1e-14))

#draw and fit the ratio
c3 = root.TCanvas(f"c3",f"c3",400,400)
h_ratio_ccbar = root.TH1D("h_ratio_ccbar","h_ratio_ccbar",len(mass_ratio_ccbar),array('d',hbd_paper_ccbar_mass_edges))
for i in range(len(mass_ratio_ccbar)):
    h_ratio_ccbar.SetBinContent(i+1, ratio_ccbar_mine_hbd[i])
h_ratio_ccbar_star200 = root.TH1D("h_ratio_ccbar_star200","h_ratio_ccbar_star200",len(mass_ratio_star200),array('d',star200_paper_ccbar_mass_edges))
for i in range(len(mass_ratio_star200)):
    h_ratio_ccbar_star200.SetBinContent(i+1, ratio_ccbar_mine_star200[i])
    h_ratio_ccbar_star200.SetBinError(i+1, 0.4*ratio_ccbar_mine_star200[i]) #assume 10% error
h_ratio_ccbar_star200.SetMarkerStyle(20)
h_ratio_ccbar_star200.SetMarkerColor(root.kRed+1)
h_ratio_ccbar_star200.SetLineColor(root.kRed+1)

Format_Hist_total(h_ratio_ccbar,"m_{ee}, GeV","Ratio of ccbar yield",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
h_ratio_ccbar.SetMinimum(0)
h_ratio_ccbar.Draw("")
h_ratio_ccbar_star200.Draw("same p")
fit_ratio_ccbar = root.TF1("fit_ratio_ccbar","pol3",0,5)
h_ratio_ccbar.Fit(fit_ratio_ccbar,"QR")
fit_ratio_ccbar.Draw("same")

fit_ratio_ccbar_star200 = root.TF1("fit_ratio_ccbar_star200","pol3",0.5,5)
h_ratio_ccbar_star200.Fit(fit_ratio_ccbar_star200,"QR")
fit_ratio_ccbar_star200.Draw("same")
c3.Draw()
print("const double ccbar_yield_ratio_fit_params[4] = {",fit_ratio_ccbar.GetParameter(0),",",fit_ratio_ccbar.GetParameter(1),",",fit_ratio_ccbar.GetParameter(2),",",fit_ratio_ccbar.GetParameter(3),"};")
print("const double ccbar_yield_ratio_fit_params[4] = {",fit_ratio_ccbar_star200.GetParameter(0),",",fit_ratio_ccbar_star200.GetParameter(1),",",fit_ratio_ccbar_star200.GetParameter(2),",",fit_ratio_ccbar_star200.GetParameter(3),"};")

const double ccbar_yield_ratio_fit_params[4] = { 1.8331257272264028 , -2.9515104807885386 , 2.1291795033921224 , -0.37004672594005783 };
const double ccbar_yield_ratio_fit_params[4] = { 2.7769208445602707 , -2.5369613285880557 , 1.2541555292690372 , -0.21169598438949042 };


In [34]:
import numpy as np

# --- Digitized c\bar{c} -> ee from right panel (sqrt(sNN)=27 GeV) ---
# M_ee in GeV/c^2 ; dN/dM in (c^2/GeV)
mee_ccbar_star = np.array([
 0.08139, 0.22093, 0.35465, 0.51744, 0.66860, 0.79651, 0.98255, 1.16279, 1.33139, 1.53488, 1.71511, 1.84883, 1.96511, 2.09883, 2.23837, 2.38953, 2.59302
], dtype=float)

dndm = np.array([
     0.00005359282227808751, 0.00014309339789044228, 0.000263480985725280, 0.0003008750442442845, 0.000256579469002744, 0.0002020577461377244, 0.0001321409619342679, 0.00007980264593818095, 0.00005218903308949766, 0.000028343244795582354, 0.000016668711014599265, 0.00001090094069530079, 0.0000075176255805536485, 0.000004916348393784993, 0.000003215174960508243, 0.0000019939387595524385, 0.000001112012092778792
], dtype=float)


scale_star = dnch_star if dnch_star else 1.0
y = dndm / scale_star

# ---- Variable bin edges from midpoints ----
edges_ccbar_star = np.zeros(len(mee_ccbar_star) + 1, dtype=float)
edges_ccbar_star[1:-1] = 0.5 * (mee_ccbar_star[:-1] + mee_ccbar_star[1:])
edges_ccbar_star[0]    = mee_ccbar_star[0]  - 0.5 * (mee_ccbar_star[1] - mee_ccbar_star[0])
edges_ccbar_star[-1]   = mee_ccbar_star[-1] + 0.5 * (mee_ccbar_star[-1] - mee_ccbar_star[-2])

# ---- Histogram: contents are dN/dM (not counts), so Fill() is not used ----
hist_ccbar_star = root.TH1D(
    "h_ccbar_27GeV",
    "c#bar{c} #rightarrow e^{+}e^{-} (27 GeV);M_{ee} (GeV/c^{2});dN/dM (c^{2}/GeV)",
    len(mee_ccbar_star), edges_ccbar_star
)

for ibin, val in enumerate(y, start=1):
    hist_ccbar_star.SetBinContent(ibin, float(val))
    hist_ccbar_star.SetBinError(ibin, 0.0)

# ---- Draw ----


In [35]:
canvases = []
legends =[]
for ipar in [0]:#[0,1,2,3,4,5]:
    c0 = root.TCanvas(f"c0_{ipar}",f"c0_{ipar}",1100,1100)
    c0.Divide(1,1)
    canvases.append(c0)
    iOpt = len(file_names)-1
    rebin = 1

    centr_length = [1,0.2/0.93,0.2/0.93,0.2/0.93,0.2/0.93,0.13/0.93]
    for icentr in range(ipar,ipar+1):
        legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[icentr]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()
        c0.cd(1)
        root.gPad.SetLogy()
        Format_Hist_total(hists_read[iOpt][0],"m_{e^{+}e^{-}} [GeV]","d^{2}N/dydm_{e^{+}e^{-}}#divide dN_{ch}/d#eta|_{#eta=0} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,1,3,1,0.99,1.35)
        hists_read[iOpt][1].SetFillColorAlpha(1,0.4)
        hists_read[iOpt][0].Draw("")
        hists_read[iOpt][1].Draw("same e2")
        legends.append(root.TLegend(0.25,0.83,0.99,0.89,"p_{T}^{e }>0.3 GeV, |y^{e}|<0.35"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()      

        #sum_sim_proj.Draw("same L")
        hists_read[iOpt][0].SetMaximum(2e-1)
        hists_read[iOpt][0].SetMinimum(5e-10)

        Format_Graph(hists_star,Msize=2,Mstyle=29,Lcolor=2,Mcolor=2,Lwidth=2)
        hists_star.Draw("P same")
        Format_Hist_total(hbd_paper,"m_{e^{+}e^{-}} [GeV]","(1/N_{ch}) d^{2}N/dydm_{e^{+}e^{-}} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,4,3,4,0.99,1.35)
        hbd_paper.Scale(1)
        hbd_paper.Draw("same p")
        hists_sim_porj_ccbar.SetLineColor(root.kBlack)
        hists_sim_porj_ccbar.SetLineWidth(3)
        hists_sim_porj_bbbar.SetLineColor(root.kRed+1)
        hists_sim_porj_bbbar.SetLineWidth(1)
        hists_sim_porj_thermal.SetLineColor(root.kGreen+1)
        hists_sim_porj_thermal.SetLineWidth(1)
        hists_sim_porj_ccbar.Draw("same hist")
        #hists_sim_porj_bbbar.Draw("same hist")
        #hists_sim_porj_thermal.Draw("same hist")
        #hists_sim_proj_ccbar_bbar = hists_sim_porj_ccbar.Clone("hists_sim_proj_ccbar_bbar")
        #hists_sim_proj_ccbar_bbar.Add(hists_sim_porj_bbbar)
        #hists_sim_proj_ccbar_bbar.Add(hists_sim_porj_thermal)
        #hists_sim_proj_ccbar_bbar.SetLineColor(root.kMagenta+2)
        #hists_sim_proj_ccbar_bbar.SetLineWidth(3)
        #hists_sim_proj_ccbar_bbar.Draw("same hist")
        #hists_sim_porj_phi.SetLineColor(root.kOrange+7)
        #hists_sim_porj_phi.SetLineWidth(2)
        #hists_sim_porj_phi.Draw("same hist")

        hist_ccbar_star.SetLineColor(root.kRed)
        hist_ccbar_star.SetLineWidth(3)
        hist_ccbar_star.Draw("same hist")
        hbd_paper_ccbar_hist.Scale(1.0)
        hbd_paper_ccbar_hist.SetLineColor(root.kBlue+1)
        hbd_paper_ccbar_hist.SetLineWidth(3)
        hbd_paper_ccbar_hist.Draw("same hist")
        #hbd_paper.SetMaximum(2e-1)
        gr_200.Draw("same P L")

        star200_paper_ccbar_hist.SetLineColor(root.kGreen+2)
        star200_paper_ccbar_hist.SetLineWidth(3)
        star200_paper_ccbar_hist.Draw("same hist")


        hists_read[iOpt][0].Draw("same")
        hists_read[iOpt][1].Draw("same e2")



        legends.append(root.TLegend(0.3,0.65,0.95,0.9))
        Format_Legend(legends[-1],0.04,12)
        legends[-1].AddEntry(0,"","")
        legends[-1].SetNColumns(2)
        legends[-1].AddEntry(0,"","")
        legends[-1].AddEntry(hists_read[iOpt][0],"Mine","pl")
        legends[-1].AddEntry(hbd_paper,"HBD PHENIX","pl")
        legends[-1].AddEntry(hist_ccbar_star,"STAR 27 GeV","l")
        legends[-1].AddEntry(gr_200,"STAR 200 GeV","pl")
        #legends[-1].AddEntry(hists_sim_porj_ccbar,"c#bar{c} simulation","l")
        #legends[-1].AddEntry(hbd_paper_ccbar_hist,"c#bar{c} from HBD PHENIX","l")
        #legends[-1].AddEntry(hist_ccbar_star,"c#bar{c} from STAR 27 GeV","l")
        #legends[-1].AddEntry(star200_paper_ccbar_hist,"c#bar{c} from STAR 200 GeV","l")
        #legends[-1].AddEntry(hists_star,"#splitline{STAR #sqrt{s_{NN}} = 27 GeV}{0-80% Au+Au}","pl")
        #legends[-1].AddEntry(hists_star,"STAR #sqrt{s_{NN}} = 27 GeV 0-80% Au+Au","pl")
        legends[-1].Draw()

    #if True: root.PHENIXTools.DrawPreliminary(0.6, 0.6, 0.3)


    # Draw the logo on the canvas with specified coordinates and size
    #logo.DrawImage(0.1, 0.8, 0.3, 0.9)  # Adjust the coordinates and size as needed

    c0.Draw()
    c0.Print(f"output/mass/all_corr_mass.png")

In [36]:
canvases = []
legends =[]
for ipar in [0]:#[0,1,2,3,4,5]:
    c0 = root.TCanvas(f"c0_{ipar}",f"c0_{ipar}",1000,1000)
    c0.Divide(1,1)
    canvases.append(c0)
    iOpt = len(file_names)-1
    rebin = 1

    centr_length = [1,0.2/0.93,0.2/0.93,0.2/0.93,0.2/0.93,0.13/0.93]
    for icentr in range(ipar,ipar+1):
        legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[icentr]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()
        c0.cd(1)
        root.gPad.SetLogy()
        Format_Hist_total(hists_read[iOpt][0],"m_{e^{+}e^{-}} [GeV]","dN/dm_{e^{+}e^{-}}#divide dN_{ch}/d#eta|_{#eta=0} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,1,3,1,0.99,1.35)
        hists_read[iOpt][0].Draw("")
        hists_read[iOpt][1].SetFillColorAlpha(1,0.4)
        hists_read[iOpt][1].Draw("same e2")
        legends.append(root.TLegend(0.25,0.83,0.99,0.89,"p_{T}^{e }>0.3 GeV, |y^{e}|<0.35"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()

        legends.append(root.TLegend(0.25,0.63,0.8,0.92))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].AddEntry(0,"","")
        legends[-1].AddEntry(hists_read[iOpt][0],"PHENIX","pl")
        legends[-1].AddEntry(hists_star,"#splitline{STAR #sqrt{s_{NN}} = 27 GeV}{0-80% Au+Au}","pl")
        #legends[-1].AddEntry(hists_star,"STAR #sqrt{s_{NN}} = 27 GeV 0-80% Au+Au","pl")
        legends[-1].Draw()
        


        #sum_sim_proj.Draw("same L")
        hists_read[iOpt][0].SetMaximum(2e-1)
        hists_read[iOpt][0].SetMinimum(5e-10)

        Format_Graph(hists_star,Msize=2,Mstyle=29,Lcolor=2,Mcolor=2,Lwidth=2)
        hists_star.Draw("P same")
        Format_Hist_total(hbd_paper,"m_{e^{+}e^{-}} [GeV]","(1/N_{ch}) dN/dm_{e^{+}e^{-}} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,4,3,4,0.99,1.35)
        hbd_paper.Scale(1)
        #hbd_paper.Draw("same p")
        hists_sim_porj_ccbar.SetLineColor(root.kBlue+1)
        hists_sim_porj_ccbar.SetLineWidth(1)
        hists_sim_porj_bbbar.SetLineColor(root.kRed+1)
        hists_sim_porj_bbbar.SetLineWidth(1)
        hists_sim_porj_thermal.SetLineColor(root.kGreen+1)
        hists_sim_porj_thermal.SetLineWidth(1)
        hists_sim_porj_ccbar.Draw("same hist")
        hists_sim_porj_bbbar.Draw("same hist")
        hists_sim_porj_thermal.Draw("same hist")
        hists_sim_proj_ccbar_bbar = hists_sim_porj_ccbar.Clone("hists_sim_proj_ccbar_bbar")
        hists_sim_proj_ccbar_bbar.Add(hists_sim_porj_bbbar)
        hists_sim_proj_ccbar_bbar.Add(hists_sim_porj_thermal)
        hists_sim_proj_ccbar_bbar.SetLineColor(root.kMagenta+2)
        hists_sim_proj_ccbar_bbar.SetLineWidth(3)
        hists_sim_proj_ccbar_bbar.Draw("same hist")
        #hists_sim_porj_phi.SetLineColor(root.kOrange+7)
        #hists_sim_porj_phi.SetLineWidth(2)
        #hists_sim_porj_phi.Draw("same hist")

        hist_ccbar_star.SetLineColor(root.kCyan+2)
        hist_ccbar_star.SetLineWidth(2)
        hist_ccbar_star.Draw("same hist")
        hbd_paper_ccbar_hist.Scale(1.0)
        hbd_paper_ccbar_hist.SetLineColor(root.kMagenta+3)
        hbd_paper_ccbar_hist.SetLineWidth(2)
        #hbd_paper_ccbar_hist.Draw("same hist")
        #hbd_paper.SetMaximum(2e-1)
        gr_200.Draw("same P L")

    #if True: root.PHENIXTools.DrawPreliminary(0.6, 0.6, 0.3)


    # Draw the logo on the canvas with specified coordinates and size
    #logo.DrawImage(0.1, 0.8, 0.3, 0.9)  # Adjust the coordinates and size as needed

    c0.Draw()
    #c0.Print(f"output/mass/mee_star.pdf")